In [78]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
from datetime import datetime

# importing csv files.
crimes_19 = pd.read_csv('input_data/Chicago_Crimes_2019.csv')
crimes_20 = pd.read_csv('input_data/Chicago_Crimes_2020.csv')
crimes_21 = pd.read_csv('input_data/Chicago_Crimes_2021.csv')

# merging crimes data for the last 3 years into a signle dataframe
crimes_data = pd.merge((pd.merge(crimes_19,crimes_20, how="outer")),crimes_21,how="outer")

print(len(crimes_data))

679438


In [75]:
# converting Date column in dataframe to date format and adding columns for month, year, day, time
crimes_data["Date"] = pd.to_datetime(crimes_data['Date'])
crimes_data["Day"] = crimes_data['Date'].dt.day_name()
crimes_data["Month"] = crimes_data['Date'].dt.month_name()
crimes_data["Year"] = crimes_data['Date'].dt.year
crimes_data["Time"] =crimes_data['Date'].dt.time
crimes_data["AM_PM"] = pd.to_datetime(crimes_data['Date']).dt.strftime("%p")

crimes_data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Y Coordinate,Updated On,Latitude,Longitude,Location,Day,Month,Year,Time,AM_PM
0,11715789,JC297664,2019-01-01 14:00:00,021XX W BELMONT AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,False,False,...,NaN,6/9/2019 16:05,NaN,NaN,NaN,Tuesday,January,2019,14:00:00,PM
1,11554737,JC102447,2019-01-01 20:00:00,077XX S ABERDEEN ST,051A,ASSAULT,AGGRAVATED - HANDGUN,STREET,False,True,...,1853584.0,3/30/2020 15:48,41.753720,-87.651399,"(41.753720143, -87.65139884)",Tuesday,January,2019,20:00:00,PM
2,11552577,JC100040,2019-01-01 00:31:00,032XX W LAWRENCE AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESTAURANT,False,False,...,1931709.0,1/10/2019 15:16,41.968444,-87.709342,"(41.968444497, -87.709341738)",Tuesday,January,2019,00:31:00,AM
3,11552587,JC100034,2019-01-01 00:05:00,006XX E 83RD PL,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,...,1849762.0,1/10/2019 15:16,41.742968,-87.608410,"(41.742968219, -87.6084099)",Tuesday,January,2019,00:05:00,AM
4,11552596,JC100045,2019-01-01 00:03:00,001XX W HURON ST,430,BATTERY,AGGRAVATED: OTHER DANG WEAPON,HOTEL/MOTEL,False,False,...,1905043.0,1/10/2019 15:16,41.894822,-87.632134,"(41.894821547, -87.632133928)",Tuesday,January,2019,00:03:00,AM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679433,12352092,JE172277,2021-03-18 10:45:00,006XX W OHARE ST,2093,NARCOTICS,FOUND SUSPECT NARCOTICS,GOVERNMENT BUILDING / PROPERTY,False,False,...,NaN,4/13/2022 16:50,NaN,NaN,NaN,Thursday,March,2021,10:45:00,AM
679434,12357360,JE153789,2021-02-27 09:07:00,042XX W MADISON ST,2014,NARCOTICS,MANUFACTURE / DELIVER - HEROIN (WHITE),VEHICLE NON-COMMERCIAL,False,False,...,NaN,4/13/2022 16:50,NaN,NaN,NaN,Saturday,February,2021,09:07:00,AM
679435,12669473,JF206525,2021-03-30 00:00:00,078XX S MUSKEGON AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,APARTMENT,False,False,...,NaN,4/13/2022 16:50,NaN,NaN,NaN,Tuesday,March,2021,00:00:00,AM
679436,12350271,JE165805,2021-03-11 09:16:00,036XX W LEXINGTON ST,2014,NARCOTICS,MANUFACTURE / DELIVER - HEROIN (WHITE),RESIDENCE - YARD (FRONT / BACK),False,False,...,NaN,4/13/2022 16:50,NaN,NaN,NaN,Thursday,March,2021,09:16:00,AM
